In [ ]:
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from common.utils import PyConfig
from ml.e2e_train import autoregress_sequense, batch_sequense
from ml.e2e_train import E2ETrain
from trade.backtest import launch as backtest_launch
from loguru import logger # type: ignore
from copy import deepcopy

%reload_ext autoreload
%autoreload 2

logger.remove()
logger.add(sys.stderr, format="{level} {message}", colorize=True, level="DEBUG")

def save_model(model):
    model.eval()
    example = torch.rand((1,) + model.inp_shape)
    model_jit = torch.jit.trace(model, example)
    model_jit.save("model.pt")

    # torch.save(model, "model.pt")

In [ ]:
cfg = PyConfig("configs/maw.py").get_inference()
# cfg.date_start = "2008.01.01"
# cfg.date_end = "2018.01.01"

In [ ]:
e2e_train = E2ETrain(cfg)
e2e_train.load_data(dataset_root="/Users/andrybin/Yandex.Disk.localized/fin_data", val_size=0.2)

In [ ]:
DEVICE = torch.device("cpu")
model = e2e_train.train(num_epochs=1500, 
                        resume=True,
                        device=DEVICE)

In [ ]:
model.eval()
p = e2e_train.train_sets[cfg.symbol].price
with torch.no_grad():
    seq_result = batch_sequense(model, 
                                p, 
                                e2e_train.train_sets[cfg.symbol].features, 
                                cfg.fee_rate, 
                                device=DEVICE)
    output_seq = seq_result.model_ans.cpu().numpy()
    result_seq = seq_result.profit_curve().cpu().numpy()
    fee_seq = seq_result.fees.cumsum(0).cpu().numpy()
hold = e2e_train.compute_hold(p, norm=True)[0]

fig, ax1 = plt.subplots(2, 1, height_ratios = [3, 1], figsize=(10, 8))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(result_seq)
# ax1.plot(fee_seq)
ax1.plot(hold, linewidth=3)
# ax1.plot(e2e_train.p - e2e_train.p[0], linewidth=3)

ax1.plot([p.shape[0]*(1-e2e_train.val_size), p.shape[0]*(1-e2e_train.val_size)], 
         [0, result_seq.max()],
         "--",
         linewidth=3)
ax1.grid("on")
plt.tight_layout()
plt.grid("on")

ax2 = ax1.twinx()
ax2.bar(list(range(output_seq.shape[0])), height=output_seq, width=1, alpha=0.4)

ax1 = plt.subplot(2, 1, 2)
ax1.fill(np.hstack([result_seq - hold, np.zeros(1)]))

In [ ]:
DEVICE = torch.device("cpu")

cfg = PyConfig("zz.py").test()
date_start = datetime.strptime(cfg.date_start, '%Y-%m-%dT%H:%M:%S').strftime('%m/%d/%Y')
date_end = datetime.strptime(cfg.date_end, '%Y-%m-%dT%H:%M:%S').strftime('%m/%d/%Y')

years = [pd.to_datetime(d).date().strftime('%Y-%m-%dT%H:%M:%S') for d in 
         pd.date_range(start=date_start, end=date_end, freq="Y")]

date_start = cfg.date_start
last_prof = 0
for year in years[19:-1]:
    cfg = PyConfig("zz.py").test()
    cfg.date_end = year
    e2e_train = E2ETrain(cfg)
    e2e_train.load_data()
    model = e2e_train.train(num_epochs=200, resume=False, device=DEVICE)
    save_model(model)
    
    cfg = PyConfig("zz.py").test()
    cfg.date_start = year
    brok_results = backtest_launch(cfg)

    plt.plot(brok_results.daily_hist.days, brok_results.daily_hist.profit + last_prof, linewidth=1, color="b", alpha=0.6)
    last_prof += brok_results.final_profit
    plt.grid("on")
    plt.tight_layout()
    # legend.append(f"{cfg.date_start}-{cfg.date_end}")

In [ ]:
brok_results = backtest(cfg)
